In [5]:
import pandas as pd

file_paths = [
    r"D:\Proyectos\amag\fine_tuning\oo_finetuning_VF.xlsx",
    r"D:\Proyectos\amag\fine_tuning\orders_outcomes_GPT.xlsx",
]
# Read and concatenate all CSV files into a single DataFrame
df = pd.concat([pd.read_excel(file) for file in file_paths], ignore_index=True)

In [6]:
df['outcome_ALEXIS'].value_counts()

6. Irrelevante                     139
1. Demandante                      113
2. Demandado                        38
4. Abandonado por el Demandante     20
Name: outcome_ALEXIS, dtype: int64

In [7]:
def convert_to_custom_format(dataset):
    converted_data = []
    for _, row in dataset.iterrows():
        json_dict = {
            "messages": [
                {"role": "system", "content": "Clasifica el siguiente fragmento de una decisión judicial en sólo una de las siguientes seis categorías: '1. Demandante', cuando la decisión es favorable al demandante, porque se declara fundada la pretensión, o se dictan medidas de protección a su favor; '2. Demandado', cuando la decisión es favorable al demandado, ya sea porque se desestima la pretensión del demandante, se declara infundada la pretensión, existe una nulidad, se declara la improcedencia o no se dictan medidas de protección; '4. Abandonado por el Demandante', cuando el caso ha concluido porque el demandante no ha asistido al proceso o ha mostrado desinterés en su resultado; '6. Irrelevante' cuando no es posible precisar el sentido del fallo. Sólo escribe el nombre de la categoría. Fragmento de la decisión judicial:"},
                {"role": "user", "content": row['input']},
                {"role": "assistant", "content": row['outcome_ALEXIS']}
            ]
        }
        converted_data.append(json_dict)
    return converted_data

converted_data = convert_to_custom_format(df)

In [8]:
from sklearn.model_selection import train_test_split

# Stratified splitting. Assuming 'Top Category' can be used for stratification
train_data, val_data = train_test_split(
    converted_data,
    test_size=0.2,
    stratify=df['outcome_ALEXIS'],
    random_state=42  # for reproducibility
)

In [9]:
import json  
def write_to_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for entry in data:
            json.dump(entry, file)
            file.write('\n')

training_file_name = "train_oo_2.jsonl"
validation_file_name = "val_oo_2.jsonl"

write_to_jsonl(train_data, training_file_name)
write_to_jsonl(val_data, validation_file_name)

In [10]:
import openai # make sure to have newest version installed; pip install --upgrade openai
openai.api_key
n_tokens = 0
from openai import OpenAI
client = OpenAI(api_key= openai.api_key)

In [11]:
data_path = r"D:\Proyectos\amag\fine_tuning\train_oo_2.jsonl"
print(data_path)
create_file_data_path = client.files.create(file=open(data_path, "rb"),purpose="fine-tune")

D:\Proyectos\amag\fine_tuning\train_oo_2.jsonl


In [12]:
create_file_data_path

FileObject(id='file-4gFxu0LEB6e7SBzV6lREoWTl', bytes=778243, created_at=1725821946, filename='train_oo_2.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [13]:
training_file_modes_family = [['file-4gFxu0LEB6e7SBzV6lREoWTl']]
for training_file_modes in [training_file_modes_family[0]]:
    for training_file_mode in training_file_modes:
             client.fine_tuning.jobs.create(
             training_file=training_file_mode,
             model="gpt-4o-mini-2024-07-18",
             hyperparameters={"n_epochs":3})

In [18]:
client.fine_tuning.jobs.list(limit=2)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-2WHS3wguwkGsjFSFJy84Qo9d', created_at=1725822055, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:de-jure::A5HgtIRG', finished_at=1725823005, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-K76iiXW5odTuDCttkbsOHqcT', result_files=['file-PcHLK2FzhXOduqvjHoiJVPbl'], seed=295173108, status='succeeded', trained_tokens=541950, training_file='file-4gFxu0LEB6e7SBzV6lREoWTl', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)], object='list', has_more=False)

In [19]:
import pandas as pd

df = pd.read_csv(r'D:\Proyectos\amag\classification\amb_orders_77k.csv')
sampled_df = df.groupby('law_field').apply(lambda x: x.sample(n=min(len(x), 5)).reset_index(drop=True))
sampled_df = sampled_df.drop(columns=['opinion_text'])
sampled_df

case_file  \
law_field                                               
CIVIL                  0   00285-2018-0-1601-JR-CI-01   
                       1   00077-2019-0-0201-JP-CI-02   
                       2   00069-2020-0-1201-JR-CI-02   
                       3   00041-2018-0-2506-JM-CI-02   
                       4   00046-2018-0-1213-JR-CI-01   
COMERCIAL              0   00007-2022-0-1504-JP-CO-01   
                       1   00012-2020-0-1501-JR-CO-04   
                       2   00065-2020-0-1706-JR-CO-03   
                       3   00005-2022-0-1501-JR-CO-03   
                       4   00016-2022-0-1504-JP-CO-01   
CONTENCIOSO ADM.       0   00196-2018-0-1801-SP-CA-01   
                       1   00004-2018-0-1305-JM-CA-01   
                       2   00264-2019-0-1801-SP-CA-01   
                       3   00074-2018-0-1801-SP-CA-02   
                       4   00028-2019-0-2110-JM-CA-01   
DERECHO CONSTITUCIONAL 0   00002-2019-0-0401-JR-DC-01   
                       1   00005-2022-0-0401-JR-DC-01   
                       2   00015-2019-0-0401-JR-DC-01   
                       3   00010-2019-0-1801-JR-DC-01   
                       4  00028-2020-19-1801-SP-DC-01   
FAMILIA CIVIL          0   00001-2021-0-1411-JR-FC-02   
                       1   00491-2020-0-2004-JR-FC-01   
                       2   01559-2018-0-1408-JR-FC-01   
                       3   00218-2018-0-1401-JR-FC-02   
                       4   00629-2018-0-1408-JR-FC-02   
FAMILIA TUTELAR        0   00069-2022-0-0602-JR-FT-01   
                       1   00045-2021-0-0222-JM-FT-01   
                       2   00365-2021-0-2001-JR-FT-01   
                       3   00096-2021-0-2202-JM-FT-01   
                       4   00089-2020-0-3398-JR-FT-02   
LABORAL                0  00498-2018-46-2802-JR-LA-01   
                       1   00175-2018-0-1801-SP-LA-10   
                       2   01314-2020-0-1801-JP-LA-07   
                       3   00033-2019-0-2208-JP-LA-01   
                       4   00028-2022-0-2601-JR-LA-01   

                                                                id  \
law_field                                                            
CIVIL                  0  documentoD.html?nid=DSIrWqIAxhCADmeMfAcy   
                       1       documentoD.html?nid=fLuEEEwMzIugLJc   
                       2     documentoD.html?nid=RydYvTIVWjqMEekcF   
                       3      documentoD.html?nid=dUPbqPYCToLEcYkQ   
                       4      documentoD.html?nid=kKssoYdKazEjsIHm   
COMERCIAL              0     documentoD.html?nid=rZgWLGdVapMVcNsCT   
                       1    documentoD.html?nid=jieZabmuSMkFUGPGMo   
                       2    documentoD.html?nid=aLWZGuipJbntpwGkJS   
                       3       documentoD.html?nid=McEgUFzlvDwOtqk   
                       4    documentoD.html?nid=ZYxfBwhsTcnUqYMuFA   
CONTENCIOSO ADM.       0       documentoD.html?nid=etHMQuMCsRSAiHb   
                       1      documentoD.html?nid=nqWQbKMRNsWdqzSG   
                       2  documentoD.html?nid=bYWnhBGKEVQYOlCYUZhd   
                       3      documentoD.html?nid=ZSOTffejIPVMpueh   
                       4    documentoD.html?nid=rvvJkuFbmJoCYISQjH   
DERECHO CONSTITUCIONAL 0       documentoD.html?nid=DPnKrxwMuiUhnap   
                       1    documentoD.html?nid=YGdpPZsSYgDlkEZaiM   
                       2   documentoD.html?nid=ldKvnfOMDmjZkFCpvnV   
                       3      documentoD.html?nid=DVpbeFfKYbPlgFxh   
                       4       documentoD.html?nid=HxhFJnbTTEobtOy   
FAMILIA CIVIL          0     documentoD.html?nid=rRhMOaYXACKxEuORh   
                       1      documentoD.html?nid=lvSiYFduOqOXDFQn   
                       2  documentoD.html?nid=ezRZoKTBnuJWbzqxQpaV   
                       3    documentoD.html?nid=wsJbOuamttkQSJjvWK   
                       4       documentoD.html?nid=qIkEtYyFhPSLAxr   
FAMILIA TUTELAR        0   documentoD.html?n

In [20]:
client = OpenAI(api_key=)

def generate_output(text):
    try:
        # Create chat completion
        completion = client.chat.completions.create(
            model="ft:gpt-4o-mini-2024-07-18:de-jure::A5HgtIRG",
            messages=[
                {"role": "system", "content": "Clasifica el siguiente fragmento de una decisión judicial en sólo una de las siguientes seis categorías: '1. Demandante', cuando la decisión es favorable al demandante, porque se declara fundada la pretensión, o se dictan medidas de protección a su favor; '2. Demandado', cuando la decisión es favorable al demandado, ya sea porque se desestima la pretensión del demandante, se declara infundada la pretensión, existe una nulidad, se declara la improcedencia o no se dictan medidas de protección; '4. Abandonado por el Demandante', cuando el caso ha concluido porque el demandante no ha asistido al proceso o ha mostrado desinterés en su resultado; '6. Irrelevante' cuando no es posible precisar el sentido del fallo. Sólo escribe el nombre de la categoría. Fragmento de la decisión judicial:"},
                {"role": "user", "content": text}
            ]
        )
        return completion.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")

sampled_df['outcome_GPT'] = sampled_df['input'].apply(generate_output)
sampled_df.to_excel(r'D:\Proyectos\amag\classification\oo_accuracy_2.xlsx', index=False)